In [1]:
%%px
from utils.creds import *
load_sf_creds()
username=os.environ["user"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="TEST_WH"
database=os.environ["dbname"]
schema="TPCH_SF1"
import sys
import bodo
rank=bodo.get_rank()
log = open(f"/tmp/process{rank}.log", "w")
sys.stderr = log

Starting 8 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

In [2]:
%%px
import time
import bodo
import bodosql
@bodo.jit(cache=False)
def tpch_q01_sql(schema,conn_str):
    t1 = time.time()
    bc = bodosql.BodoSQLContext(
        {
            "lineitem": bodosql.TablePath(
                f"{schema}.lineitem", "sql", conn_str=conn_str, reorder_io=True
            ),
        })
    total = bc.sql(
        """select
                l_returnflag,
                l_linestatus,
                sum(l_quantity) as sum_qty,
                sum(l_extendedprice) as sum_base_price,
                sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
                sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
                avg(l_quantity) as avg_qty,
                avg(l_extendedprice) as avg_price,
                avg(l_discount) as avg_disc,
                count(*) as count_order
            from
                lineitem
            where
                l_shipdate <= date '1998-12-01' - interval '90' day
            group by
                l_returnflag,
                l_linestatus
            order by
                l_returnflag,
                l_linestatus"""
    )

    print("Q01 Execution time (s): ", time.time() - t1)
    return total

q1_result=tpch_q01_sql(schema,f"snowflake://{username}:{password}@{account}/{database}/PUBLIC?warehouse={warehouse}")

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] Q01 Execution time (s):  4.328776076930808


In [3]:
%%px
q1_result

,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,37734107.0,5.658655e+10,5.375826e+10,5.590907e+10,25.522006,38273.129735,0.049985,1478493
1,N,F,991417.0,1.487505e+09,1.413082e+09,1.469649e+09,25.516472,38284.467761,0.050093,38854
2,N,O,74476040.0,1.117017e+11,1.061182e+11,1.103670e+11,25.502227,38249.117989,0.049997,2920374
3,R,F,37719753.0,5.656804e+10,5.374129e+10,5.588962e+10,25.505794,38250.854626,0.050009,1478870


Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 18762, 'identifier': 'ipcontroller-1657129475-owwi-18276'}
Stopping engine(s): 1657129476


In [ ]:
%%px
from utils.creds import *
load_aws_creds()
import time
import bodo
import bodosql
@bodo.jit(cache=True)
def divvy_sql():
    t1 = time.time()
    bc = bodosql.BodoSQLContext(
        {
            "divvy": bodosql.TablePath(
                "s3://bodo-divvy-data/parquet", "parquet"
            ),
        })
    df = bc.sql(
        """select
               member_casual,count(*) as `count` from divvy group by member_casual"""
    )
    print("Execution time (s): ", time.time() - t1)
    return df

df=divvy_sql()

%px:   0%|          | 0/24 [00:00<?, ?tasks/s]

[stdout:0] Execution time (s):  0.7086308506204659


In [ ]:
%%px
df

,member_casual,count


,member_casual,count


,member_casual,count


,member_casual,count


,member_casual,count


...

,member_casual,count


,member_casual,count
0,casual,1376114


,member_casual,count
1,member,1866825


,member_casual,count


,member_casual,count


In [ ]:
%%px
from utils.creds import *
load_sf_creds()
username=os.environ["user"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="TEST_WH"
database=os.environ["dbname"]
schema="TPCH_SF10"
import sys
import bodo
rank=bodo.get_rank()
log = open(f"/tmp/process{rank}.log", "w")
sys.stderr = log

Starting 24 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/24 [00:00<?, ?engine/s]

In [ ]:
%%px
import time
import bodo
import bodosql
@bodo.jit(cache=True)
def tpch_q01_sql(schema,conn_str):
    t1 = time.time()
    bc = bodosql.BodoSQLContext(
        {
            "lineitem": bodosql.TablePath(
                f"{schema}.lineitem", "sql", conn_str=conn_str, reorder_io=True
            ),
        })
    total = bc.sql(
        """select
                l_returnflag,
                l_linestatus,
                sum(l_quantity) as sum_qty,
                sum(l_extendedprice) as sum_base_price,
                sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
                sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
                avg(l_quantity) as avg_qty,
                avg(l_extendedprice) as avg_price,
                avg(l_discount) as avg_disc,
                count(*) as count_order
            from
                lineitem
            where
                l_shipdate <= date '1998-12-01' - interval '90' day
            group by
                l_returnflag,
                l_linestatus
            order by
                l_returnflag,
                l_linestatus"""
    )

    print("Q01 Execution time (s): ", time.time() - t1)
    return total

q1_result=tpch_q01_sql(schema,f"snowflake://{username}:{password}@{account}/{database}/PUBLIC?warehouse={warehouse}")

%px:   0%|          | 0/24 [00:00<?, ?tasks/s]

[stdout:0] Q01 Execution time (s):  7.744855493318028


In [ ]:
%%px
q1_result

In [ ]:
%%px
from utils.creds import *
load_aws_creds()
import time
import bodo
import bodosql
@bodo.jit(cache=True)
def divvy_sql():
    t1 = time.time()
    bc = bodosql.BodoSQLContext(
        {
            "divvy": bodosql.TablePath(
                "s3://bodo-divvy-data/parquet", "parquet"
            ),
        })
    df = bc.sql(
        """select
               member_casual,count(*) as `count` from divvy group by member_casual"""
    )
    print("Execution time (s): ", time.time() - t1)
    return df

df=divvy_sql()

%px:   0%|          | 0/24 [00:00<?, ?tasks/s]

[stdout:0] Execution time (s):  0.7086308506204659


In [ ]:
%%px
df

,member_casual,count


,member_casual,count


,member_casual,count


,member_casual,count


,member_casual,count


...

,member_casual,count


,member_casual,count
0,casual,1376114


,member_casual,count
1,member,1866825


,member_casual,count


,member_casual,count
